In [1]:
import sympy
import numpy as np
from tqdm import tqdm
import dill
dill.settings["recurse"] = True
from sympy import Rational as R

# Necessary symbols

In [2]:
(
    reference_x,
    reference_y,
    physical_x_0,
    physical_y_0,
    physical_x_1,
    physical_y_1,
    physical_x_2,
    physical_y_2,
) = sympy.symbols("x y x_0 y_0 x_1 y_1 x_2 y_2")

rotation_matrix = sympy.Matrix([[+R(0), +R(1)], [-R(1), +R(0)]])

In [3]:
t0_hat = sympy.Matrix([-R(1), +R(1)]) / sympy.sqrt(2)
t1_hat = sympy.Matrix([+R(0), +R(1)])
t2_hat = sympy.Matrix([+R(1), +R(0)])

n0_hat = rotation_matrix @ t0_hat
n1_hat = rotation_matrix @ t1_hat
n2_hat = rotation_matrix @ t2_hat

In [4]:
t0_sub_hat = sympy.Matrix([R(+1, 3), R(+1, 3)]) / sympy.sqrt(R(2, 9))
t1_sub_hat = sympy.Matrix([R(-2, 3), R(+1, 3)]) / sympy.sqrt(R(5, 9))
t2_sub_hat = sympy.Matrix([R(+1, 3), R(-2, 3)]) / sympy.sqrt(R(5, 9))

n0_sub_hat = rotation_matrix @ t0_sub_hat
n1_sub_hat = rotation_matrix @ t1_sub_hat
n2_sub_hat = rotation_matrix @ t2_sub_hat

In [5]:
v0 = {reference_x: 0, reference_y: 0}
v1 = {reference_x: 1, reference_y: 0}
v2 = {reference_x: 0, reference_y: 1}
barycenter_hat = {reference_x: R(1, 3), reference_y: R(1, 3)}

e0 = {reference_x: R(1, 2), reference_y: R(1, 2)}
e1 = {reference_x: 0, reference_y: R(1, 2)}
e2 = {reference_x: R(1, 2), reference_y: 0}

e0_sub = {reference_x: R(1, 6), reference_y: R(1, 6)}
e1_sub = {reference_x: R(2, 3), reference_y: R(1, 6)}
e2_sub = {reference_x: R(1, 6), reference_y: R(2, 3)}

In [6]:
subtriangle_0_equation = (reference_y >= 0)&(reference_y <= reference_x)&(2*reference_y <= 1 - reference_x)
subtriangle_1_equation = (reference_y <= 1 - reference_x)&(reference_y >= -2*reference_x + 1)&(2*reference_y >= 1 - reference_x)
subtriangle_2_equation = (reference_x >= 0)&(reference_y <= -2*reference_x + 1)&(reference_y >= reference_x)

# Get basis functions

In [7]:
monomial_basis = sympy.Matrix(
[
 1,
 reference_x,
 reference_y,
 reference_x**2,
 reference_y**2,
 reference_x*reference_y,
 reference_x**3,
 reference_y**3,
 reference_x**2*reference_y,
 reference_x*reference_y**2,
 ]
)

In [8]:
Vander = sympy.Matrix(np.zeros((30, 30)))

for i in range(10):
            
    # subtriangle 0    
    Vander[00+i, 0] = monomial_basis[i].subs(v0)
    Vander[00+i, 1] = monomial_basis[i].diff(reference_x).subs(v0)
    Vander[00+i, 2] = monomial_basis[i].diff(reference_y).subs(v0)
    
    # subtriangle 1
    Vander[10+i, 3] = monomial_basis[i].subs(v1)
    Vander[10+i, 4] = monomial_basis[i].diff(reference_x).subs(v1)
    Vander[10+i, 5] = monomial_basis[i].diff(reference_y).subs(v1)
    
    # subtriangle 2
    Vander[20+i, 6] = monomial_basis[i].subs(v2)
    Vander[20+i, 7] = monomial_basis[i].diff(reference_x).subs(v2)
    Vander[20+i, 8] = monomial_basis[i].diff(reference_y).subs(v2)    
    
    # Normal derivatives at master triangle edge midpoints
    Vander[10+i, 9] =  (n0_hat[0]*monomial_basis[i].diff(reference_x) + n0_hat[1]*monomial_basis[i].diff(reference_y)).subs(e0)
    Vander[20+i, 10] = (n1_hat[0]*monomial_basis[i].diff(reference_x) + n1_hat[1]*monomial_basis[i].diff(reference_y)).subs(e1)
    Vander[00+i, 11] = (n2_hat[0]*monomial_basis[i].diff(reference_x) + n2_hat[1]*monomial_basis[i].diff(reference_y)).subs(e2)
        
    # Continuity at vertices
    Vander[00+i, 12+0] = +monomial_basis[i].subs(v1)
    Vander[10+i, 12+0] = -monomial_basis[i].subs(v1)
    Vander[00+i, 12+1] = +monomial_basis[i].diff(reference_x).subs(v1)
    Vander[10+i, 12+1] = -monomial_basis[i].diff(reference_x).subs(v1)
    Vander[00+i, 12+2] = +monomial_basis[i].diff(reference_y).subs(v1)
    Vander[10+i, 12+2] = -monomial_basis[i].diff(reference_y).subs(v1)
    
    Vander[10+i, 12+3] = +monomial_basis[i].subs(v2)
    Vander[20+i, 12+3] = -monomial_basis[i].subs(v2)
    Vander[10+i, 12+4] = +monomial_basis[i].diff(reference_x).subs(v2)
    Vander[20+i, 12+4] = -monomial_basis[i].diff(reference_x).subs(v2)
    Vander[10+i, 12+5] = +monomial_basis[i].diff(reference_y).subs(v2)
    Vander[20+i, 12+5] = -monomial_basis[i].diff(reference_y).subs(v2)
    
    Vander[20+i, 12+6] = +monomial_basis[i].subs(v0)
    Vander[00+i, 12+6] = -monomial_basis[i].subs(v0)
    Vander[20+i, 12+7] = +monomial_basis[i].diff(reference_x).subs(v0)
    Vander[00+i, 12+7] = -monomial_basis[i].diff(reference_x).subs(v0)
    Vander[20+i, 12+8] = +monomial_basis[i].diff(reference_y).subs(v0)
    Vander[00+i, 12+8] = -monomial_basis[i].diff(reference_y).subs(v0)
    
    # Continuity at barycenter    
    Vander[10+i, 12+9] =  +monomial_basis[i].subs(barycenter_hat)    
    Vander[00+i, 12+9] =  -monomial_basis[i].subs(barycenter_hat)
    Vander[10+i, 12+10] = +monomial_basis[i].diff(reference_x).subs(barycenter_hat)
    Vander[00+i, 12+10] = -monomial_basis[i].diff(reference_x).subs(barycenter_hat)
    Vander[10+i, 12+11] = +monomial_basis[i].diff(reference_y).subs(barycenter_hat)
    Vander[00+i, 12+11] = -monomial_basis[i].diff(reference_y).subs(barycenter_hat)
        
    Vander[20+i, 12+12] = +monomial_basis[i].subs(barycenter_hat)
    Vander[00+i, 12+12] = -monomial_basis[i].subs(barycenter_hat)
    Vander[20+i, 12+13] = +monomial_basis[i].diff(reference_x).subs(barycenter_hat)
    Vander[00+i, 12+13] = -monomial_basis[i].diff(reference_x).subs(barycenter_hat)
    Vander[20+i, 12+14] = +monomial_basis[i].diff(reference_y).subs(barycenter_hat)
    Vander[00+i, 12+14] = -monomial_basis[i].diff(reference_y).subs(barycenter_hat)        
    
    
    # Continuity at edges of subtriangles
    Vander[00+i, 12+15] = +(n0_sub_hat[0]*monomial_basis[i].diff(reference_x) + n0_sub_hat[1]*monomial_basis[i].diff(reference_y)).subs(e0_sub) 
    Vander[20+i, 12+15] = -(n0_sub_hat[0]*monomial_basis[i].diff(reference_x) + n0_sub_hat[1]*monomial_basis[i].diff(reference_y)).subs(e0_sub)
           
    Vander[10+i, 12+16] = +(n1_sub_hat[0]*monomial_basis[i].diff(reference_x) + n1_sub_hat[1]*monomial_basis[i].diff(reference_y)).subs(e1_sub)
    Vander[00+i, 12+16] = -(n1_sub_hat[0]*monomial_basis[i].diff(reference_x) + n1_sub_hat[1]*monomial_basis[i].diff(reference_y)).subs(e1_sub)
            
    Vander[20+i, 12+17] = +(n2_sub_hat[0]*monomial_basis[i].diff(reference_x) + n2_sub_hat[1]*monomial_basis[i].diff(reference_y)).subs(e2_sub)
    Vander[10+i, 12+17] = -(n2_sub_hat[0]*monomial_basis[i].diff(reference_x) + n2_sub_hat[1]*monomial_basis[i].diff(reference_y)).subs(e2_sub)

In [9]:
hct_basis = Vander.inv()@sympy.diag(monomial_basis, monomial_basis, monomial_basis)

# Get mapping function

In [10]:
monomial_basis = sympy.Matrix(
    [
        1,
        reference_x,
        reference_y,
    ]
)

V = sympy.zeros(3, 3)

for dofidx, basis in enumerate(monomial_basis):

    V[dofidx, 0] = basis.subs({reference_x: 0, reference_y: 0})
    V[dofidx, 1] = basis.subs({reference_x: 1, reference_y: 0})
    V[dofidx, 2] = basis.subs({reference_x: 0, reference_y: 1})

mapping_basis = V.inv() @ monomial_basis

global_x = (
      mapping_basis[0] * physical_x_0
    + mapping_basis[1] * physical_x_1
    + mapping_basis[2] * physical_x_2
)
global_y = (
      mapping_basis[0] * physical_y_0
    + mapping_basis[1] * physical_y_1
    + mapping_basis[2] * physical_y_2
)

mapping_function = sympy.Matrix([global_x, global_y])

# Necessary symbols for transformations

In [11]:
J_cofactor_T = mapping_function.jacobian([reference_x, reference_y]).cofactor_matrix().T
J = J_cofactor_T / J_cofactor_T.det()
J_inv = J.inv()

In [ ]:
barycenter = sympy.Matrix([
                            R(1, 3)*(physical_x_0 + physical_x_1 + physical_x_2),
                            R(1, 3)*(physical_y_0 + physical_y_1 + physical_y_2)
                            ])

In [ ]:
G0_hat = n0_hat.row_join(t0_hat).T
G1_hat = n1_hat.row_join(t1_hat).T
G2_hat = n2_hat.row_join(t2_hat).T

G0_sub_hat = n0_sub_hat.row_join(t0_sub_hat).T
G1_sub_hat = n1_sub_hat.row_join(t1_sub_hat).T
G2_sub_hat = n2_sub_hat.row_join(t2_sub_hat).T


l0 = ((physical_x_2 - physical_x_1) ** 2 + (physical_y_2 - physical_y_1) ** 2) ** R(1, 2)
l1 = ((physical_x_2 - physical_x_0) ** 2 + (physical_y_2 - physical_y_0) ** 2) ** R(1, 2)
l2 = ((physical_x_1 - physical_x_0) ** 2 + (physical_y_1 - physical_y_0) ** 2) ** R(1, 2)

l0_sub = ((barycenter[0] - physical_x_0) ** 2 + (barycenter[1] - physical_y_0) ** 2) ** R(1, 2)
l1_sub = ((barycenter[0] - physical_x_1) ** 2 + (barycenter[1] - physical_y_1) ** 2) ** R(1, 2)
l2_sub = ((barycenter[0] - physical_x_2) ** 2 + (barycenter[1] - physical_y_2) ** 2) ** R(1, 2)


t0 = sympy.Matrix([physical_x_2 - physical_x_1, physical_y_2 - physical_y_1]) / l0
t1 = sympy.Matrix([physical_x_2 - physical_x_0, physical_y_2 - physical_y_0]) / l1
t2 = sympy.Matrix([physical_x_1 - physical_x_0, physical_y_1 - physical_y_0]) / l2

t0_sub = sympy.Matrix([barycenter[0] - physical_x_0, barycenter[1] - physical_y_0]) / l0_sub
t1_sub = sympy.Matrix([barycenter[0] - physical_x_1, barycenter[1] - physical_y_1]) / l1_sub
t2_sub = sympy.Matrix([barycenter[0] - physical_x_2, barycenter[1] - physical_y_2]) / l2_sub

n0 = rotation_matrix @ t0
n1 = rotation_matrix @ t1
n2 = rotation_matrix @ t2

n0_sub = rotation_matrix @ t0_sub
n1_sub = rotation_matrix @ t1_sub
n2_sub = rotation_matrix @ t2_sub

G0 = n0.row_join(t0).T
G1 = n1.row_join(t1).T
G2 = n2.row_join(t2).T

G0_sub = n0_sub.row_join(t0_sub).T
G1_sub = n1_sub.row_join(t1_sub).T
G2_sub = n2_sub.row_join(t2_sub).T

B0 = G0_hat @ J_inv.T @ G0.T
B1 = G1_hat @ J_inv.T @ G1.T
B2 = G2_hat @ J_inv.T @ G2.T

B0_sub = G0_sub_hat @ J_inv.T @ G0_sub.T
B1_sub = G1_sub_hat @ J_inv.T @ G1_sub.T
B2_sub = G2_sub_hat @ J_inv.T @ G2_sub.T


f_interpolation_extended = sympy.Matrix(sympy.symbols("f_0:15"))

f_interpolation = sympy.Matrix(f_interpolation_extended[:12])

f_interpolation[9] =  n0[0] * f_interpolation_extended[9] +  n0[1] * f_interpolation_extended[10]
f_interpolation[10] = n1[0] * f_interpolation_extended[11] + n1[1] * f_interpolation_extended[12]
f_interpolation[11] = n2[0] * f_interpolation_extended[13] + n2[1] * f_interpolation_extended[14]

### $V^{c}$

In [ ]:
V_c = sympy.diag(
    R(1),
    J_inv.T,
    R(1),
    J_inv.T,
    R(1),
    J_inv.T,
    B0,
    B1,
    B2,
    R(1),
    sympy.eye(2, 2),
    R(1),
    sympy.eye(2, 2),    
    R(1),
    sympy.eye(2, 2),    
    R(1),
    sympy.eye(2, 2),    
    R(1),
    sympy.eye(2, 2),    
    R(1),
    R(1),
    R(1),
    
)

### $E$

In [ ]:
E = np.zeros((30, 33), dtype=int)
for i in range(30):
    for j in range(33):
        if (i <= 9) and (j <= 9) and (i == j):
            E[i][j] = 1
        elif (i == 10) and (j == 11):
            E[i][j] = 1
        elif (i == 11) and (j == 13):
            E[i][j] = 1        
        elif (11 < i) and (j + 1 < 33) and (j == i + 1):
            E[i][j+2] = 1
                        
E = sympy.Matrix(E)

### $D$

In [ ]:
D = E.T.copy()

In [ ]:
# Tangential derivative at e0
D[10, +6] = +R(3, 2)/l0
D[10, +3] = -R(3, 2)/l0
D[10, +7] = -R(1, 4)*t0[0]
D[10, +8] = -R(1, 4)*t0[1]
D[10, +4] = -R(1, 4)*t0[0]
D[10, +5] = -R(1, 4)*t0[1]

# Tangential derivative at e1
D[12, +6] = +R(3, 2)/l1
D[12, +0] = -R(3, 2)/l1
D[12, +7] = -R(1, 4)*t1[0]
D[12, +8] = -R(1, 4)*t1[1]
D[12, +1] = -R(1, 4)*t1[0]
D[12, +2] = -R(1, 4)*t1[1]

# Tangential derivative at e2
D[14, +3] = +R(3, 2)/l2
D[14, +0] = -R(3, 2)/l2
D[14, +4] = -R(1, 4)*t2[0]
D[14, +5] = -R(1, 4)*t2[1]
D[14, +1] = -R(1, 4)*t2[0]
D[14, +2] = -R(1, 4)*t2[1]

### $M$

In [ ]:
V = E @ V_c @ D
M_c = V.T

In [ ]:
E_nullspace = np.zeros((12, 30), dtype=int)
for i in range(12):
    for j in range(30):
        if (i == j) and (i < 12) and (j < 12):
            E_nullspace[i][j] = 1
E_nullspace = sympy.Matrix(E_nullspace)

In [ ]:
M = E_nullspace @ M_c

## Calculate bilinear form and right values for forward pushed element

In [ ]:
piecewise_hct_basis = sympy.Matrix([sympy.Piecewise(
    (bb0, subtriangle_0_equation),
    (bb1, subtriangle_1_equation),
    (bb2, subtriangle_2_equation),
    (0, True)) for bb0, bb1, bb2 in zip(hct_basis.col(0), hct_basis.col(1), hct_basis.col(2))])

In [ ]:
N = len(piecewise_hct_basis)

weak_form_functional_xxyy = [[0 for i in range(N)] for j in range(N)]
weak_form_functional_xyxy = [[0 for i in range(N)] for j in range(N)]
weak_form_right_part =      [[0 for i in range(N)] for j in range(N)]

for idx, jdx in tqdm([(idx, jdx) for idx in range(N) for jdx in range(N)]):
    
    first = piecewise_hct_basis[idx]
    second = piecewise_hct_basis[jdx]

    # -----------------------U------------------------------------------------------------------------------
    u_x = first.diff(reference_x) * J_cofactor_T.row(0)[0] + first.diff(reference_y) * J_cofactor_T.row(1)[0]
    u_y = first.diff(reference_x) * J_cofactor_T.row(0)[1] + first.diff(reference_y) * J_cofactor_T.row(1)[1]

    u_xx = u_x.diff(reference_x) * J_cofactor_T.row(0)[0] + u_x.diff(reference_y) * J_cofactor_T.row(1)[0]
    u_xy = u_x.diff(reference_x) * J_cofactor_T.row(0)[1] + u_x.diff(reference_y) * J_cofactor_T.row(1)[1]

    u_yy = u_y.diff(reference_x) * J_cofactor_T.row(0)[1] + u_y.diff(reference_y) * J_cofactor_T.row(1)[1]
    # ------------------------------------------------------------------------------------------------------

    # -----------------------V------------------------------------------------------------------------------
    v_x = second.diff(reference_x) * J_cofactor_T.row(0)[0] + second.diff(reference_y) * J_cofactor_T.row(1)[0]    
    v_y = second.diff(reference_x) * J_cofactor_T.row(0)[1] + second.diff(reference_y) * J_cofactor_T.row(1)[1]

    v_xx = v_x.diff(reference_x) * J_cofactor_T.row(0)[0] + v_x.diff(reference_y) * J_cofactor_T.row(1)[0]
    v_xy = v_x.diff(reference_x) * J_cofactor_T.row(0)[1] + v_x.diff(reference_y) * J_cofactor_T.row(1)[1]

    v_yy = v_y.diff(reference_x) * J_cofactor_T.row(0)[1] + v_y.diff(reference_y) * J_cofactor_T.row(1)[1]
    # ------------------------------------------------------------------------------------------------------

    # -----------------------bilinear-form------------------------------------------------------------------
    A = u_xx * v_xx
    B = u_xy * v_xy
    b = u_xx * v_yy
    C = u_yy * v_yy
    # ------------------------------------------------------------------------------------------------------

    weak_form_functional_xyxy[idx][jdx] = A + 2 * B + C
    weak_form_functional_xxyy[idx][jdx] = A + 2 * b + C
    weak_form_right_part[idx][jdx] = first * second
    
# -----------------------------------------------------------------------------------------------------

weak_form_functional_xxyy = sympy.Matrix(weak_form_functional_xxyy)
weak_form_functional_xyxy = sympy.Matrix(weak_form_functional_xyxy)
weak_form_right_part = sympy.Matrix(weak_form_right_part)

# The determinant of the Jacobian does not depend on (x, y) so we can take it outside to make the symbolic integration faster
weak_form_functional_xxyy = weak_form_functional_xxyy * abs(J_cofactor_T.det()) / J_cofactor_T.det()**4
weak_form_functional_xyxy = weak_form_functional_xyxy * abs(J_cofactor_T.det()) / J_cofactor_T.det()**4
weak_form_right_part = weak_form_right_part * abs(J_cofactor_T.det())

In [ ]:
lambdify_symbols = [
    reference_x, 
    reference_y,
    physical_x_0,
    physical_y_0,
    physical_x_1,
    physical_y_1,
    physical_x_2,
    physical_y_2,
]

In [ ]:
weak_form_right_part = M @ weak_form_right_part @ M.T @ f_interpolation
weak_form_right_part_lambdified = sympy.lambdify([*lambdify_symbols, *f_interpolation_extended], weak_form_right_part, cse=True)

In [ ]:
weak_form_functional_xxyy_lambdified = sympy.lambdify(lambdify_symbols, M @ weak_form_functional_xxyy @ M.T, cse=True)
weak_form_functional_xyxy_lambdified = sympy.lambdify(lambdify_symbols, M @ weak_form_functional_xyxy @ M.T, cse=True)

In [ ]:
quadrature_weights = np.array([
0.090817990382754,
0.036725957756467,
0.036725957756467,
0.036725957756467,
0.072757916845420,
0.072757916845420,
0.072757916845420,
0.072757916845420,
0.072757916845420,
0.072757916845420,
0.028327242531057,
0.028327242531057,
0.028327242531057,
0.028327242531057,
0.028327242531057,
0.028327242531057,
0.009421666963733,
0.009421666963733,
0.009421666963733,
0.009421666963733,
0.009421666963733,
0.009421666963733,
0.045321059435528,
0.045321059435528,
0.045321059435528,
]) / 2


quadrature_points = np.array([
[0.333333333333333,0.333333333333333],
[0.485577633383657,0.028844733232685],
[0.485577633383657,0.485577633383657],
[0.028844733232685,0.485577633383657],
[0.141707219414880,0.550352941820999],
[0.307939838764121,0.141707219414880],
[0.550352941820999,0.307939838764121],
[0.307939838764121,0.550352941820999],
[0.141707219414880,0.307939838764121],
[0.550352941820999,0.141707219414880],
[0.025003534762686,0.728323904597411],
[0.246672560639903,0.025003534762686],
[0.728323904597411,0.246672560639903],
[0.246672560639903,0.728323904597411],
[0.025003534762686,0.246672560639903],
[0.728323904597411,0.025003534762686],
[0.009540815400299,0.923655933587500],
[0.066803251012200,0.009540815400299],
[0.923655933587500,0.066803251012200],
[0.066803251012200,0.923655933587500],
[0.009540815400299,0.066803251012200],
[0.923655933587500,0.009540815400299],
[0.109481575485037,0.781036849029926],
[0.109481575485037,0.109481575485037],
[0.781036849029926,0.109481575485037],
])

In [ ]:
def weak_form_functional_xxyy_lambdified_quadrature(x_0, y_0, x_1, y_1, x_2, y_2):
    
    return weak_form_functional_xxyy_lambdified(*quadrature_points.T, x_0, y_0, x_1, y_1, x_2, y_2)@quadrature_weights

In [ ]:
def weak_form_functional_xyxy_lambdified_quadrature(x_0, y_0, x_1, y_1, x_2, y_2):
    
    return weak_form_functional_xyxy_lambdified(*quadrature_points.T, x_0, y_0, x_1, y_1, x_2, y_2)@quadrature_weights

In [ ]:
def weak_form_right_part_lambdified_quadrature(
                                               x_0, y_0, 
                                               x_1, y_1, 
                                               x_2, y_2,
                                                
                                               f_0, f_1, f_2, 
                                               f_3, f_4, f_5, 
                                               f_6, f_7, f_8, 
                                               f_9, f_10, f_11, 
                                               f_12, f_13, f_14
                                               ):
    
    return weak_form_right_part_lambdified(
                                            *quadrature_points.T, 
                                            x_0, y_0, 
                                            x_1, y_1, 
                                            x_2, y_2, 
                                            
                                            f_0, f_1, f_2, 
                                            f_3, f_4, f_5, 
                                            f_6, f_7, f_8, 
                                            f_9, f_10, f_11, 
                                            f_12, f_13, f_14
                                            )@quadrature_weights

In [ ]:
mhct = M@hct_basis

piecewise_hct_basis = sum(sympy.Piecewise(
    (bb0, subtriangle_0_equation),
    (bb1, subtriangle_1_equation),
    (bb2, subtriangle_2_equation),
    (0, True))*fj for fj, bb0, bb1, bb2 in zip(f_interpolation, mhct.col(0), mhct.col(1), mhct.col(2)))

f_interpolation_normal = sympy.Matrix(sympy.symbols("f_0:12"))

piecewise_hct_basis_normal = sum(sympy.Piecewise(
    (fj*bb0, subtriangle_0_equation),
    (fj*bb1, subtriangle_1_equation),
    (fj*bb2, subtriangle_2_equation),
    (0, True)) for fj, bb0, bb1, bb2 in zip(f_interpolation_normal, mhct.col(0), mhct.col(1), mhct.col(2)))

In [ ]:
interpolation_function = sympy.lambdify([*lambdify_symbols, *f_interpolation_extended], piecewise_hct_basis, cse=True)

interpolation_function_dx = sympy.lambdify(
                                             [*lambdify_symbols, *f_interpolation_extended],
                                             J.T[0, 0]*piecewise_hct_basis.diff(reference_x) + J.T[0, 1]*piecewise_hct_basis.diff(reference_y), 
                                             cse=True
                                        )

interpolation_function_dy = sympy.lambdify(
                                             [*lambdify_symbols, *f_interpolation_extended],
                                             J.T[1, 0]*piecewise_hct_basis.diff(reference_x) + J.T[1, 1]*piecewise_hct_basis.diff(reference_y), 
                                             cse=True
                                        )

choices = {
                "f":interpolation_function,
                "f_x":interpolation_function_dx,
                "f_y":interpolation_function_dy,                
           }                       

In [ ]:
def interpolation_function_lambdified(
                                        x, y, 
                                        x_0, y_0, 
                                        x_1, y_1, 
                                        x_2, y_2, 
                                        
                                        f_0, f_1, f_2, 
                                        f_3, f_4, f_5, 
                                        f_6, f_7, f_8, 
                                        f_9, f_10, 
                                        f_11, f_12, 
                                        f_13, f_14, 
                                        which
                                    ): 
    return choices[which](
                            x, y, 
                            x_0, y_0, 
                            x_1, y_1, 
                            x_2, y_2, 
                            
                            f_0, f_1, f_2, 
                            f_3, f_4, f_5, 
                            f_6, f_7, f_8, 
                            f_9, f_10, 
                            f_11, f_12, 
                            f_13, f_14
                        )

In [ ]:
interpolation_function_normal = sympy.lambdify([*lambdify_symbols, *f_interpolation_normal], piecewise_hct_basis_normal, cse=True)

interpolation_function_normal_dx = sympy.lambdify(
                                             [*lambdify_symbols, *f_interpolation_normal],
                                             J.T[0, 0]*piecewise_hct_basis_normal.diff(reference_x) + J.T[0, 1]*piecewise_hct_basis_normal.diff(reference_y), 
                                             cse=True
                                        )

interpolation_function_normal_dy = sympy.lambdify(
                                             [*lambdify_symbols, *f_interpolation_normal],
                                             J.T[1, 0]*piecewise_hct_basis_normal.diff(reference_x) + J.T[1, 1]*piecewise_hct_basis_normal.diff(reference_y), 
                                             cse=True
                                        )

choices_normal = {
                "f":interpolation_function_normal,
                "f_x":interpolation_function_normal_dx,
                "f_y":interpolation_function_normal_dy,                
           }                       

In [ ]:
def interpolation_function_normal_lambdified(
                                        x, y, 
                                        x_0, y_0, 
                                        x_1, y_1, 
                                        x_2, y_2, 
                                        
                                        f_0, f_1, f_2, 
                                        f_3, f_4, f_5, 
                                        f_6, f_7, f_8, 
                                        f_9, f_10, f_11,
                                        
                                        which
                                    ): 
    return choices_normal[which](
                            x, y, 
                            x_0, y_0, 
                            x_1, y_1, 
                            x_2, y_2, 
                            
                            f_0, f_1, f_2, 
                            f_3, f_4, f_5, 
                            f_6, f_7, f_8, 
                            f_9, f_10, f_11
                        )

In [ ]:
mapping_function_lambdified = sympy.lambdify(
    [*lambdify_symbols], mapping_function[:, 0], cse=True
)

### Save the results

In [ ]:
dill.dump(
    weak_form_functional_xxyy_lambdified_quadrature,
    open(
        "../calculations/macro_cubic_hct_weak_form_functional_xxyy", "wb"
    ),
)
dill.dump(
    weak_form_functional_xyxy_lambdified_quadrature,
    open(
        "../calculations/macro_cubic_hct_weak_form_functional_xyxy", "wb"
    ),
)
dill.dump(
    weak_form_right_part_lambdified_quadrature,
    open("../calculations/macro_cubic_hct_weak_form_right_part", "wb"),
)

dill.dump(
    interpolation_function_lambdified,
    open("../calculations/macro_cubic_hct_basis", "wb"),
)

dill.dump(
    interpolation_function_normal_lambdified,
    open("../calculations/macro_cubic_hct_basis_normal", "wb"),
)

dill.dump(
    mapping_function_lambdified,
    open("../calculations/macro_cubic_hct_mapping_function", "wb"),
)